# Classification of reddit user posts using graph machine learning with Stellargraph

We apply the graph machine algorithm APPNP [1] to the task of classifying reddit user posts into 41 different categories using the dataset published in [2] which can be downloaded [here](https://www.kaggle.com/manoelribeiro/hateful-users-on-twitter/home).  

The following is a description of the dataset [2]:

>Reddit is a large online discussion forum where users post and comment on content in different topical
>communities. We constructed a graph dataset from Reddit posts made in the month of September, 2014. The node label in this case is the community, or “subreddit”, that a post belongs to. We sampled
>50 large communities and built a post-to-post graph, connecting posts if the same user comments
>on both. In total this dataset contains 232,965 posts with an average degree of 492. We use the first
>20 days for training and the remaining days for testing (with 30% used for validation). For features,
>we use off-the-shelf 300-dimensional GloVe CommonCrawl word vectors [3]; for each post, we
>concatenated (i) the average embedding of the post title, (ii) the average embedding of all the post’s
>comments (iii) the post’s score, and (iv) the number of comments made on the post.


We demonstrate the advantage of using graph features and the scalability of the APPNP algorithm for node classification on the reddit dataset.  We first train a MLP on the node features and then propagate this model using APPNP.  Training is only done on the node features, this approach allows model training to be completed in under a 1 minute on an 8th gen quad-core i7.


**References**

1. Predict then propagate: Graph neural networks meet personalized pagerank. J. Klicpera,  A. Bojchevski, & S. Günnemann arxiv:1810.05997, 2018.


2. Inductive Representation Learning on Large Graphs. W.L. Hamilton, R. Ying, and J. Leskovec arXiv:1706.02216 [cs.SI], 2017.


3. Glove: Global vectors for word representation. J. Pennington, R. Socher, and C. D. Manning. In EMNLP, 2014.

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import os

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegressionCV

import stellargraph as sg
from stellargraph.mapper import GraphSAGENodeGenerator, FullBatchNodeGenerator
from stellargraph.layer import GraphSAGE, GCN, GAT, APPNP
from stellargraph.layer.appnp import APPNPPropagationLayer
from stellargraph import globalvar
from stellargraph.core.utils import GCN_Aadj_feats_op

from tensorflow.keras import layers, optimizers, losses, metrics, Model, models
from sklearn import preprocessing, feature_extraction
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.sparse import coo_matrix

import matplotlib.pyplot as plt
import seaborn as sns
import json

%matplotlib inline

## Loading the Reddit Dataset

First, we load the reddit dataset which is stored as a series of json files.  We first load the graph data and then the node features and labels and ensure that indexing is consistent across the graph, labels, and node features.

In [3]:
data_dir = os.path.expanduser("~/data/reddit")

In [4]:
with open(os.path.join(data_dir, "reddit-G.json")) as gfile:
    graph_data = json.load(gfile)
    
list_node_ids = list(d['id'] for d in graph_data['nodes'])

adj_list = [(link['source'], link['target']) for link in graph_data['links']]

gnx = nx.Graph()
gnx.add_edges_from(adj_list)

In [5]:
print("Number of nodes:", gnx.number_of_nodes())
print("Number of edges:", gnx.number_of_edges())

Number of nodes: 231443
Number of edges: 11606919


In [4]:
with open(os.path.join(data_dir, "reddit-class_map.json")) as tfile:
    labels = json.load(tfile)

feats = np.load(data_dir + "/reddit-feats.npy")

feats[:,0] = np.log(feats[:,0]+1.0)
feats[:,1] = np.log(feats[:,1]-min(np.min(feats[:,1]), -1))

scaler = preprocessing.StandardScaler()
feats = scaler.fit_transform(feats)

feat_id_map = json.load(open(data_dir + "/reddit-id_map.json"))

# sort node features to match the order of feat_id_map
sorted_idxs = np.array([feat_id_map[key] for key in list_node_ids])
feats = feats[sorted_idxs,:]

#sort node labnels to match the order of feat_id_map
labels = np.array([labels[key] for key in list_node_ids])

In [5]:
node_data = pd.DataFrame(feats)

target_encoding = preprocessing.OneHotEncoder(sparse=False, categories='auto')
targets = target_encoding.fit_transform(labels.reshape(-1, 1))
targets = pd.DataFrame(targets)

In [7]:
G = sg.StellarGraph(gnx, node_features=node_data)

gnx.clear()

In [8]:
train_data, test_data, train_targets, test_targets = train_test_split(
    node_data, targets, train_size=0.4, random_state=42)

val_data, test_data, val_targets, test_targets = train_test_split(
    test_data, test_targets, train_size=0.2, random_state=42)

In [10]:
generator = FullBatchNodeGenerator(G, method="gcn", sparse=True)
test_gen = generator.flow(test_data.index, test_targets)

## Training

We now create a MLP and train on the node features.

In [11]:
in_layer = layers.Input(shape=(train_data.shape[-1]))
layer = layers.Dense(128, activation='relu', kernel_regularizer="l2")(in_layer)
layer = layers.Dropout(0.5)(layer)
layer = layers.Dense(128, activation='relu', kernel_regularizer="l2")(in_layer)
layer = layers.Dropout(0.5)(layer)

#note the dimension of the output should equal the number of classes to predict!
layer = layers.Dense(train_targets.shape[-1], activation='softmax')(layer)

fully_connected_model = Model(inputs=in_layer, outputs=layer)

fully_connected_model.compile(
    optimizer=optimizers.Adam(lr=0.0001),
    loss=losses.categorical_crossentropy,
    metrics=["acc"],
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
history = fully_connected_model.fit(
    train_data, train_targets,
    epochs = 40,
    validation_data = (val_data, val_targets),
    batch_size = 300,
    verbose=0
)

The MLP attains an accuracy of ~67% on the test set.

In [18]:
test_metrics = fully_connected_model.evaluate(test_data, test_targets)
print("\nTest Set Metrics:")
for name, val in zip(fully_connected_model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

111093/111093 [==============================] - 3s 29us/sample - loss: 1.3785 - acc: 0.6729

Test Set Metrics:
	loss: 1.3785
	acc: 0.6729


## APPNP Propagation

We now create a stellargraph generator.  We use this to create an APPNP model and propagate the MLP.  No further training is happening in this step.  We then test the propagated model on the test data.

In [15]:
appnp = APPNP(layer_sizes=[train_targets.shape[-1]], 
              activations=['relu'], 
              bias=True,
              generator=generator, 
              teleport_probability=0.9, 
              dropout=0.5, 
              kernel_regularizer='l2'
)

x_inp, x_out = appnp.propagate_model(fully_connected_model)
predictions = layers.Softmax()(x_out)

propagated_model = Model(inputs=x_inp, outputs=predictions)
propagated_model.compile(loss='categorical_crossentropy', metrics=['acc'],
                  optimizer='Adam')

In [19]:
test_metrics = propagated_model.evaluate_generator(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(propagated_model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))


Test Set Metrics:
	loss: 1.8850
	acc: 0.8035


Propagating the MLP with APPNP increases the test set accuracy by ~15% without any further training.  